<a href="https://colab.research.google.com/github/BDfriends/transformer-slt/blob/master/transformer_slt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/BDfriends/transformer-slt.git

Cloning into 'transformer-slt'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 270 (delta 69), reused 203 (delta 24), pack-reused 0
Receiving objects: 100% (270/270), 7.32 MiB | 6.32 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [2]:
%cd transformer-slt

/content/transformer-slt


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 47 kB 5.7 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.0
    Uninstalling torchtext-0.13.0:
      Successfully uninstalled torchtext-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.1 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.30.0 which is incompatible.
prophet 1.1 requires tqdm>=4.36.1, but you have tqdm 4.30.0 which is incompatible.
panel 0.12.1 requires tqdm>=4.48.0, but you have tqdm 4.30.0 which is incompatible.


In [4]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating OpenNMT_py.egg-info
writing OpenNMT_py.egg-info/PKG-INFO
writing dependency_links to OpenNMT_py.egg-info/dependency_links.txt
writing entry points to OpenNMT_py.egg-info/entry_points.txt
writing requirements to OpenNMT_py.egg-info/requires.txt
writing top-level names to OpenNMT_py.egg-info/top_level.txt
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
adding license file 'LICENSE.md'
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/onmt
copying onmt/opts.py -> build/lib/onmt
copying onmt/train_single.py -> build/lib/onmt
copying onmt/model_builder.py -> build/lib/onmt
copying onmt/trainer.py -> build/lib/onmt
copying onmt/__init__.py -> build/lib/onmt
creating build/lib/onmt/bin
copying onmt/bin/train.py -> build/lib/onmt/bin
copying onmt/bin/average_models.py -> build/

In [5]:
!onmt_preprocess -train_src data/phoenix2014T.train.gloss -train_tgt data/phoenix2014T.train.de -valid_src data/phoenix2014T.dev.gloss -valid_tgt data/phoenix2014T.dev.de -save_data data/dgs -lower 

[2022-07-30 07:56:01,508 INFO] Extracting features...
[2022-07-30 07:56:01,508 INFO]  * number of source features: 0.
[2022-07-30 07:56:01,508 INFO]  * number of target features: 0.
[2022-07-30 07:56:01,508 INFO] Building `Fields` object...
[2022-07-30 07:56:01,508 INFO] Building & saving training data...
[2022-07-30 07:56:01,528 INFO] Building shard 0.
[2022-07-30 07:56:01,738 INFO]  * saving 0th train data shard to data/dgs.train.0.pt.
[2022-07-30 07:56:02,023 INFO]  * tgt vocab size: 2889.
[2022-07-30 07:56:02,025 INFO]  * src vocab size: 1233.
[2022-07-30 07:56:02,035 INFO] Building & saving validation data...
[2022-07-30 07:56:02,047 INFO] Building shard 0.
[2022-07-30 07:56:02,054 INFO]  * saving 0th valid data shard to data/dgs.valid.0.pt.


In [6]:
!python  train.py -data data/dgs -save_model model -keep_checkpoint 1 \
          -layers 2 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8  \
          -encoder_type transformer -decoder_type transformer -position_encoding \
          -max_generator_batches 2 -dropout 0.1 \
          -early_stopping 3 -early_stopping_criteria accuracy ppl \
          -batch_size 2048 -accum_count 3 -batch_type tokens -normalization tokens \
          -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 3000 -learning_rate 0.5 \
          -max_grad_norm 0 -param_init 0  -param_init_glorot \
          -label_smoothing 0.1 -valid_steps 100 -save_checkpoint_steps 100 \
          -world_size 1 -gpu_ranks 0

[2022-07-30 07:56:10,195 INFO]  * src vocab size = 1233
[2022-07-30 07:56:10,195 INFO]  * tgt vocab size = 2889
[2022-07-30 07:56:10,195 INFO] Building model...
[2022-07-30 07:56:14,320 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(1233, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feature

In [7]:
!python translate.py -model model_step_1400.pt -src data/phoenix2014T.test.gloss -output pred.txt -gpu 0 -replace_unk -beam_size 4

[2022-07-30 08:01:17,758 INFO] Translating shard 0.
/content/transformer-slt/onmt/translate/beam_search.py:187: UserWarning: An output with one or more elements was resized since it had shape [120], which does not match the required output shape [30, 4]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/content/transformer-slt/onmt/translate/beam_search.py:187: UserWarning: An output with one or more elements was resized since it had shape [48], which does not match the required output shape [12, 4]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inpla

In [8]:
# BLEU-1,2,3,4
!python tools/bleu.py 1 pred.txt data/phoenix2014T.test.de
!python tools/bleu.py 2 pred.txt data/phoenix2014T.test.de
!python tools/bleu.py 3 pred.txt data/phoenix2014T.test.de
!python tools/bleu.py 4 pred.txt data/phoenix2014T.test.de

# ROUGE
!python tools/rouge.py pred.txt data/phoenix2014T.test.de

# METEOR
!python tools/meteor.py pred.txt data/phoenix2014T.test.de

0.4604190793188173
0.342523215600003
0.2697733069312829
0.2228712296673685
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Traceback (most recent call last):
  File "tools/rouge.py", line 806, in <module>
    stemming=True)
  File "tools/rouge.py", line 89, in __init__
    Rouge.load_wordnet_db(ensure_compatibility)
  File "tools/rouge.py", line 119, in load_wordnet_db
    raise FileNotFoundError("The file '{}' does not exist".format(filepath))
FileNotFoundError: The file 'tools/wordnet_key_value.txt' does not exist
[nltk_data] Downloading package wordnet to /root/nltk_data...
Traceback (most recent call last):
  File "tools/meteor.py", line 14, in <module>
    scores = [nltk.meteor([t.lower()], p.lower()) for t,p in zip(target, pred)]
  File "tools/meteor.py", line 14, in <listcomp>
    scores = [nltk.meteor([t.lower()], p.lower()) for t,p in zip(target, pred)]
  File "/usr/local/lib/python3.7/dist-packages/nltk/translate/meteo